In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import geopandas as gpd
from shapely.geometry import Point
import glob
from skimage.morphology import dilation, square
from matplotlib.colors import LinearSegmentedColormap
import cartopy.feature as cfeature
from itertools import product
import sys
import cftime

In [ ]:
root = '/lustre/gmeteo/WORK/DATA/CORDEX-FPS-URB-RCC/nextcloud/'
dest = '/lustre/gmeteo/WORK/diezsj/research/cordex-fps-urb-rcc/results/'
ucdb_info = gpd.read_file(root  + 'CORDEX-CORE-WG/GHS_FUA_UCD/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg')

rcms = {
  'AFR-22' : [
    'REMO2015',
    'RegCM4-7',
  ],
  'AUS-22' : [
    'REMO2015',
    'RegCM4-7',
  ],
  'CAM-22' : [
    'REMO2015',
    'RegCM4-7',
  ],
  'SAM-22' : [
    'REMO2015',
    'RegCM4-7',
  ],
  'WAS-22' : [
    'REMO2015',
    'RegCM4-7',
  ],
  'EUR-11': [
    'REMO2015',
    'RegCM4-6',
  ],
  'EAS-22': [
#    'RegCM4-0', # No opendap access
    'REMO2015'
  ],
  'SEA-22' : [
    'REMO2015',
    'RegCM4-7',
  ],
  'NAM-22': [
    'RegCM4_v4-4-rc8',
    'REMO2015',
  ]
}

# coordinates name
rlat_names = {'rlat', 'y' }
rlon_names = {'rlon', 'x'}

rotated_pole_names = {'rotated_pole', 'rotated_latitude_longitude', 'Lambert_Conformal',
                     'oblique_mercator', 'crs'}

# Dictionary containing city locations and their respective domains
location = {
     'Mexico City' : dict(lon=-99.0833, lat=19.4667, domain = 'CAM-22'),
     'Buenos Aires' : dict(lon=-58.416, lat=-34.559, domain = 'SAM-22'),
     'New York' : dict(lon=-74.2261, lat=40.8858, domain = 'NAM-22'),
     'Sydney' : dict(lon=151.01810, lat=-33.79170, domain = 'AUS-22'),
     'Beijing' : dict(lon=116.41, lat=39.90, domain = 'EAS-22'),
     'Tokyo' : dict(lon = 139.84, lat = 35.65, domain = 'EAS-22'),
     'Jakarta' : dict(lon = 106.81, lat = -6.2, domain = 'SEA-22'), 
     'Johannesburg' : dict(lon=28.183, lat=-25.733, domain = 'AFR-22'),
     'Riyadh' : dict(lon=46.73300, lat=24.7000, domain = 'WAS-22'),
     'Berlin' : dict(lon=13.4039, lat=52.4683, domain = 'EUR-11'),
     'Paris' : dict(lon=  2.35, lat=48.85, domain = 'EUR-11'),
     'London' : dict(lon= -0.13, lat=51.50, domain = 'EUR-11'),
     'Madrid' : dict(lon= -3.70, lat=40.42, domain = 'EUR-11'),
     'Los Angeles': dict(lon = -118.24, lat = 34.05, domain = 'NAM-22'),
     'Montreal': dict(lon = -73.56, lat = 45.50, domain = 'NAM-22'),
     'Chicago': dict(lon = -87.55, lat = 41.73, domain = 'NAM-22'),
     'Bogota': dict(lon = -74.06, lat = 4.62, domain = 'SAM-22'),
     'Baghdad': dict(lon = 44.40, lat = 33.34, domain = 'WAS-22'),
     'Tehran': dict(lon = 51.42, lat = 35.69, domain = 'WAS-22'),
     'Tashkent': dict(lon = 69.24, lat = 41.31, domain = 'WAS-22'),
     'Cairo': dict(lon = 31.25, lat = 30.06, domain = 'AFR-22'),
     'Delhi [New Delhi]': dict(lon = 77.22, lat = 28.64, domain = 'WAS-22'),
    'Barcelona': dict(lon = 2.18, lat = 41.39, domain = 'EUR-11'),
    'Rome': dict(lon =  12.51, lat = 41.89, domain = 'EUR-11'),
    'Athens': dict(lon =   23.72, lat =  37.98, domain = 'EUR-11'),

}

model_dict={
     'REMO' : dict(urban_variable='urban',orog_variable='orog',sea_variable='sftlf'),
    'RegCM' : dict(urban_variable='sftuf',orog_variable='orog',sea_variable='sftlf'),
    }

In [ ]:
def data_city(city: str, location: dict, model: str, model_dict:dict, data: str = "CORDEX-CORE"):
    '''
    Retrieves urban, orography, sea masks, and temp dataset for the chosen city and model.

    Parameters:
        city (str): The chosen city.
        location (dict): Dictionary containing city locations and their respective domains.
        model (str): The chosen model. Options are "REMO" and "RegCM".
        data (str): The chosen data options: CORDEX-CORE or CORDEX-EUR-11.

    Returns:
        tuple: Dataset of minimal temperature, sea mask, orography mask, urban fraction, latitude, longitude of the city, and nearby stations.
    '''
    # Retrieve city data from location dictionary
    city_data = location.get(city, {})
    lon = city_data.get('lon')
    lat = city_data.get('lat')
    domain = city_data.get('domain')

    if data=="CORDEX-CORE":
        ds=xr.open_dataset("/lustre/gmeteo/WORK/DATA/C3S-CDS/C3S-CICA-Atlas/v1/CORDEX-CORE/historical/tn_CORDEX-CORE_historical_mon_197001-200512.nc")
    elif domain=="EUR-11" and data=="CORDEX-EUR-11":
        ds=xr.open_dataset("/lustre/gmeteo/WORK/DATA/C3S-CDS/C3S-CICA-Atlas/v1/CORDEX-EUR-11/historical/tn_CORDEX-EUR-11_historical_mon_197001-200512.nc")

    # Paths for sea, orography, and urban masks based on model and domain
    if model=="REMO":
        base_path_sea='/lustre/gmeteo/WORK/DATA/CORDEX-FPS-URB-RCC/nextcloud/CORDEX-CORE-WG/REMO/land-sea-mask_C/'
        base_path_urban='/lustre/gmeteo/WORK/DATA/CORDEX-FPS-URB-RCC/nextcloud/CORDEX-CORE-WG/REMO/urbanfraction_C/orig_v3/'
        base_path_orography='/lustre/gmeteo/WORK/DATA/CORDEX-FPS-URB-RCC/nextcloud/CORDEX-CORE-WG/REMO/orography_C/'
        # Searching for files
        file_sea = glob.glob(base_path_sea +'*'+domain+'*')
        file_urban = glob.glob(base_path_urban +'*'+domain+'*')
        file_orography = glob.glob(base_path_orography+'*'+domain+'*')

    elif model=="RegCM":
        base_path_sea='/lustre/gmeteo/WORK/DATA/CORDEX-FPS-URB-RCC/nextcloud/CORDEX-CORE-WG/RegCM/land-sea-mask_C/'
        base_path_urban='/lustre/gmeteo/WORK/DATA/CORDEX-FPS-URB-RCC/nextcloud/CORDEX-CORE-WG/RegCM/urbanfraction_C/'
        base_path_orography='/lustre/gmeteo/WORK/DATA/CORDEX-FPS-URB-RCC/nextcloud/CORDEX-CORE-WG/RegCM/orography_C/'
        # Searching for files
        file_sea = glob.glob(base_path_sea +'*'+domain+'*')
        file_urban = glob.glob(base_path_urban +'*'+domain+'*')
        file_orography = glob.glob(base_path_orography+ '*'+domain+'*')


    # Open datasets for sea, orography, and urban masks
    sea_mask = xr.open_dataset(file_sea[0])
    orography = xr.open_dataset(file_orography[0])
    urbanfraction = xr.open_dataset(file_urban[0])

    # Define variable names from model dictionary
    urban_variable = model_dict.get(model, {}).get('urban_variable')
    orog_variable = model_dict.get(model, {}).get('orog_variable')
    sea_variable = model_dict.get(model, {}).get('sea_variable')
                 
    ds_urban = urbanfraction[urban_variable]
    ds_orog = orography[orog_variable]
    ds_sftlf = sea_mask[sea_variable]  

    res = int(orography.attrs['CORDEX_domain'].split('-')[1])
    
    ghcnd_stations_url = 'https://www.ncei.noaa.gov/data/global-historical-climatology-network-daily/doc/ghcnd-stations.txt'
    ghcnd_stations_column_names = ['code', 'lat', 'lon', 'elev', 'name', 'net', 'numcode']
    ghcnd_stations_column_widths = [   11,     9,    10,      7,     34,     4,       10 ]
    df = pd.read_fwf(ghcnd_stations_url, header = 0, widths = ghcnd_stations_column_widths, names = ghcnd_stations_column_names)
    ghcnd_stations=gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs = 'EPSG:4326')
    
    rval = ghcnd_stations.assign(dist = ghcnd_stations.distance(Point(lon, lat)))
    rval.sort_values(by = 'dist', inplace = True)
    rval = rval[rval.dist < 0.5].to_crs(epsg=3857)  
    return ds, ds_sftlf, ds_orog, ds_urban, lat, lon, rval,res
        

In [ ]:
def select_name(names, avail_names):
  # Select variable/coordinate names among a list of potential names.
  # Potential names are matched against those available in the data set.
  return(list(names.intersection(list(avail_names)))[0])

In [ ]:
def plot_boundary(ax, lon, lat, dist_lon, dist_lat,color='blue',zorder=1,linewidth=1):
    """
    Plot the boundary of a square defined by longitude and latitude values.

    Parameters:
        ax (matplotlib.axes.Axes): The matplotlib axes to plot on.
        lon (float): Longitude value.
        lat (float): Latitude value.
        dist_lon (float): Distance in longitude.
        dist_lat (float): Distance in latitude.
        color (str, optional): Color of the boundary line.
    """
    ax.plot([lon - dist_lon, lon + dist_lon, lon + dist_lon, lon - dist_lon, lon - dist_lon],
            [lat - dist_lat, lat - dist_lat, lat + dist_lat, lat + dist_lat, lat - dist_lat],
            color=color, linewidth=linewidth,zorder=zorder)


In [ ]:
def select_non_urban_cells(urban_mask,orog_mask,sftlf_mask,urban_surrounding_mask,scale=4):
    """
    Selects non-urban cells surrounding urban areas based on dilation operation.

    Parameters:
        urban_mask (numpy.ndarray): Binary mask indicating urban areas.
        orog_mask (numpy.ndarray): Binary mask indicating orography.
        sftlf_mask (numpy.ndarray): Binary mask indicating sea areas.
        scale (int): Scaling factor to determine the extent of non-urban area selection.

    Returns:[~np.isnan(data_array)]
        numpy.ndarray: Mask of selected non-urban cells.
    """
    data_array = xr.DataArray(urban_surrounding_mask)
    kernel = np.array([[0, 1, 0],
                       [1, 1, 1],
                       [0, 1, 0]])
    
    urban_cells = np.sum(urban_mask)
    non_urban_cells = 0
    n_i = 0
    while non_urban_cells <= urban_cells * scale:
        
        # Perform dilation
        dilated_data = xr.apply_ufunc(dilation, 
                                      data_array if non_urban_cells == 0 else dilated_data, 
                                      kwargs={'footprint': kernel})
        #Delete other fixed variables (to include)
        #dilated_data = dilated_data.where(orog_mask & sftlf_mask)
        dilated_data = (dilated_data * orog_mask * sftlf_mask).astype(int)
        non_urban_cells = np.sum(dilated_data) - urban_cells
        if n_i > 10:
            print("Warning...")
            break
        n_i = n_i + 1
    
    non_urban_mask = xr.DataArray(dilated_data.where(~urban_surrounding_mask).fillna(0))
    return non_urban_mask

In [ ]:
def plot_urban_areas(ds_urban,ds_orog,ds_sftlf,urban_mask,non_urban_mask,orog_mask, sftlf_mask,urban_th, urban_elevation,orog_diff,sftlf_th,urban_area,scale):
    # Definir una paleta de colores personalizada similar a "terrain" sin azules
    colors = ['#7C5B49', '#92716B', '#A89080', '#C0B49E', '#DACCB9', '#F5F5DC']
    # Crear la paleta de colores personalizada
    custom_cmap = LinearSegmentedColormap.from_list("custom_terrain", colors)
    
    dset = orography.copy()
    lonlat = ccrs.PlateCarree()
    projvar = select_name(rotated_pole_names, dset.data_vars)
    if projvar.startswith('rotated'):
        proj = ccrs.RotatedPole(
        pole_longitude=dset[projvar].grid_north_pole_longitude,
        pole_latitude=dset[projvar].grid_north_pole_latitude
        )
    elif projvar.startswith('oblique'):
        # To be implemented soon... (https://github.com/SciTools/cartopy/pull/2096)
        proj = ccrs.ObliqueMercator(
        pole_longitude=dset[projvar].grid_north_pole_longitude,
        pole_latitude=dset[projvar].grid_north_pole_latitude
        )
    else:
        proj = ccrs.LambertConformal(
        central_longitude=dset[projvar].longitude_of_central_meridian,
        central_latitude=dset[projvar].latitude_of_projection_origin,
        standard_parallels=dset[projvar].standard_parallel
        )

                   
    fig, axes = plt.subplots(2, 3, subplot_kw={'projection': proj}, figsize=(20, 10))
    

    urban_plot = ds_urban.\
                 plot(ax=axes[0, 0], cmap='binary', 
                 vmin = np.nanmin(ds_urban), vmax = np.nanmax(ds_urban))
    axes[0, 0].set_title('Urban Fraction')
    axes[0, 0].coastlines()
    
    orography_plot = ds_orog.plot(
        ax=axes[0, 1], cmap=custom_cmap, 
        vmin = np.nanmin(ds_orog), vmax = np.nanmax(ds_orog)
    )
    axes[0, 1].set_title('Orography')
    axes[0, 1].coastlines()
    
    sea_plot = ds_sftlf.plot(
        ax=axes[0, 2], cmap='winter',
        vmin = np.nanmin(ds_sftlf), vmax = np.nanmax(ds_sftlf))
    axes[0, 2].set_title('sftlf')
    axes[0, 2].coastlines()
    
    # masked data
    urban_plot = ds_urban.where(urban_mask == 1, np.nan).plot(
        ax=axes[1, 0], cmap='binary', 
        vmin = np.nanmin(ds_urban), vmax = np.nanmax(ds_urban))
    axes[1, 0].set_title('Urban Fraction (mask >' +  str(urban_th) + ', scale = ' + str(scale) +  ')')
    axes[1, 0].coastlines()
    
    orography_plot = ds_orog.where(orog_mask == 1, np.nan).plot(
        ax=axes[1, 1], cmap=custom_cmap,
        vmin = np.nanmin(ds_orog), vmax = np.nanmax(ds_orog))
    axes[1, 1].set_title('orog (' + str(int(urban_elevation) - orog_diff) + 
                         ' m < mask > ' +  
                         str(int(urban_elevation) + orog_diff) + ' m)'
    )
    axes[1, 1].coastlines()
    
    sea_mask_plot = ds_sftlf.where(sftlf_mask == 1, np.nan).plot(
        ax=axes[1, 2], cmap='winter',
        vmin = np.nanmin(ds_sftlf), vmax = np.nanmax(ds_sftlf)
    )
    axes[1, 2].set_title('sftlf (mask >' + str(sftlf_th) + '%)')
    axes[1, 2].coastlines()
    
    
    rlon_values = orog_mask[select_name(rlon_names, orog_mask.coords)].values
    rlat_values = orog_mask[select_name(rlat_names, orog_mask.coords)].values
    dist_rlon = (rlon_values[0]- rlon_values[1])/2
    dist_rlat = (rlat_values[0]- rlat_values[1])/2 
    
    # Plot the border for cells for the differents masks
    for k in range(3):
        for i, j in product(range(np.shape(urban_mask)[0]), 
                            range(np.shape(urban_mask)[1])):

            if urban_area['urban_area'].values[i, j] == 1:
                plot_boundary(axes[1, k], rlon_values[j], rlat_values[i], 
                              dist_rlon, dist_rlat,'red',zorder=50)
                                
            elif urban_area['urban_area'].values[i, j] == 0:
                plot_boundary(axes[1, k], rlon_values[j], rlat_values[i], 
                              dist_rlon, dist_rlat,'blue',zorder=5)

    plt.subplots_adjust(wspace=0.1, hspace=0.1)  # Ajusta el espacio horizontal y vertical

    plt.savefig('urban_areas.pdf', bbox_inches='tight')

In [ ]:
def plot_cities(ds: xr.Dataset, city: str, model: str, model_dict:dict, ds_sftlf: xr.Dataset, ds_orog: xr.Dataset, ds_urban: xr.Dataset,
                CORDEX: str, lat: int, lon: int, lon_lim: int, lat_lim: int, vtmin: int, vtmax: int, rval: xr.Dataset, rlat_names:xr,rlon_names: xr, rotated_pole_names:xr, res:int,
                season: str='all',period: slice = slice('1980-01-01', '2000-12-31'),save_results:bool= True, urban_th: float = 0.1, urban_surrounding_th: float=0.1, scale:int= 4,
                orog_diff: float = 100, sftlf_th:float=70,percentil: float =10) :
    '''
    Plots urban, orography, sea masks, and mean min temperature for selected cities and models.
    
    Parameters:
    ds (xr.Dataset): Dataset containing temperature data.
    city (str): The chosen city.
    model (str): The chosen model.
    ds_sftlf (xr.Dataset): Dataset containing sea mask data.
    ds_orog (xr.Dataset): Dataset containing orography data.
    ds_urban (xr.Dataset): Dataset containing urban fraction data.
    CORDEX (str): Name of the CORDEX dataset.
    lat (int): Latitude of the city.
    lon (int): Longitude of the city.
    dist_lon (int): Distance in longitude.
    dist_lat (int): Distance in latitude.
    vtmin (int): Minimum value for temperature.
    vtmax (int): Maximum value for temperature.
    season (str, optional): The selected season for filtering temperature data. 
                            Options are "all" (default), "jfm" (January-February-March), "amj" (April-May-June), "jas" (July-August-September), and "ond" (October-November-December).
    period (slice): Slice object with datetime indices.
    rval (xr.Dataset): Dataset containing values for reference.
    orog_diff (int, optional): Maximum limit for orography. Defaults to 100.
    sftlf_th.....
    urban_th (float, optional): Minimum threshold for urban areas. Defaults to 0.1.
    percentil (float, optional): Percentile value for calculating percentiles. Defaults to 10.
    
    Returns:
    fig (plt.Figure): Plot object
    '''

    #Draws the city
    root = '/lustre/gmeteo/WORK/DATA/CORDEX-FPS-URB-RCC/nextcloud/'
    ucdb_info = gpd.read_file(root  + 'CORDEX-CORE-WG/GHS_FUA_UCD/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg')
    ucdb_city = ucdb_info.query(f'UC_NM_MN =="{city}"').to_crs(crs = 'EPSG:4326')
    if city == 'London':
        ucdb_city = ucdb_city[ucdb_city['CTR_MN_NM'] == 'United Kingdom']
    
    # Select data from the dataset based on the time range and season
    #ds = ds.sel(time=period)
    season_to_month_bounds = {
        'jfm': (1, 3),
        'amj': (4, 6),
        'jas': (7, 9),
        'ond': (10, 12)
    }
    # Select data from dataset based on time range and season 
    if season != 'all':
        lower_bound, upper_bound = season_to_month_bounds[season]
        ds= ds.sel(time=(ds['time'].dt.month >= lower_bound) & (ds['time'].dt.month <= upper_bound))
        ds= ds.sel(time=period)
    #else:
    #    ds= ds.sel(time=period)

    # Define latitude and longitude ranges based on city location and distances
    lon_min = lon - lon_lim
    if lon_min <0:  
        lon_min+=360
    lon_max = lon + lon_lim
    if lon_max <0:  
        lon_max+=360
    lat_min = lat - lat_lim
    lat_max = lat + lat_lim

    # Crop area around the city
    dlon = int(111*lon_lim/res)
    dlat = int(111*lat_lim/res)
                    
    # Create subplots with projection
    dset = orography.copy()
    lonlat = ccrs.PlateCarree()
    projvar = select_name(rotated_pole_names, dset.data_vars)
    if projvar.startswith('rotated'):
        proj = ccrs.RotatedPole(
        pole_longitude=dset[projvar].grid_north_pole_longitude,
        pole_latitude=dset[projvar].grid_north_pole_latitude
        )
    elif projvar.startswith('oblique'):
        # To be implemented soon... (https://github.com/SciTools/cartopy/pull/2096)
        proj = ccrs.ObliqueMercator(
        pole_longitude=dset[projvar].grid_north_pole_longitude,
        pole_latitude=dset[projvar].grid_north_pole_latitude
        )
    else:
        proj = ccrs.LambertConformal(
        central_longitude=dset[projvar].longitude_of_central_meridian,
        central_latitude=dset[projvar].latitude_of_projection_origin,
        standard_parallels=dset[projvar].standard_parallel
        )
    fig, axes = plt.subplots(subplot_kw={'projection': proj}, figsize=(10, 10))
    fig2, axes2 = plt.subplots(figsize=(20, 15))
    

    #Urban plot
    #latitude_mask = (ds_urban.lat >= lat_min) & (ds_urban.lat <= lat_max)
    #longitude_mask = (ds_urban.lon >= lon_min) & (ds_urban.lon <= lon_max)   
    #Plot the contour of urban data        
    #urban = ds_urban.where(latitude_mask & longitude_mask, drop=True)

    if model=="REMO":
        ds_urban['lon'][:] = ds_orog['lon']
        ds_urban['lat'][:] = ds_orog['lat']
    elif model=="RegCM":
        ds_urban = ds_urban.assign_coords(x=ds_orog.x, y=ds_orog.y)

                    
    # Urban and non-urban mask
    dist = (ds_urban['lon']-lon)**2 + (ds_urban['lat']-lat)**2
    [ilat], [ilon] = np.where(dist == np.min(dist))
    ds_urban = ds_urban.isel(**{
    select_name(rlat_names, ds_urban.coords): slice(ilat-dlat,ilat+dlat),
    select_name(rlon_names, ds_urban.coords): slice(ilon-dlon,ilon+dlon)
    })      
    urban_mask = ds_urban > urban_th
    urban_surrounding_mask = ds_urban > urban_surrounding_th
    if 'time' in urban_mask.dims:
        urban_mask = urban_mask.isel(time=0)
        urban_surrounding_mask = urban_surrounding_mask.isel(time=0)
    
    #lon_values = urban['lon'].values
    #lat_values = urban['lat'].values
    #dist_lon=(lon_values[0]- lon_values[1])/2
    #dist_lat=(lat_values[0]- lat_values[1])/2            

    # Orography mask
    urban_elevation = ds_orog.where(urban_mask).mean().item()
    dist = (ds_orog['lon']-lon)**2 + (ds_orog['lat']-lat)**2
    [ilat], [ilon] = np.where(dist == np.min(dist))
    ds_orog = ds_orog.isel(**{
    select_name(rlat_names, ds_orog.coords): slice(ilat-dlat,ilat+dlat),
    select_name(rlon_names, ds_orog.coords): slice(ilon-dlon,ilon+dlon)
    })    
    #urban_elevation = ds_orog.where(urban_mask).mean().item()
    urban_elevation_max = ds_orog.where(urban_mask).max().item()
    urban_elevation_min = ds_orog.where(urban_mask).min().item()
    
    orog_mask1 = ds_orog < (orog_diff + urban_elevation_max)
    orog_mask2 = ds_orog > (urban_elevation_min - orog_diff)
    orog_mask = orog_mask1 & orog_mask2

    rlon_values = ds_orog[select_name(rlon_names, ds_orog.coords)].values
    rlat_values = ds_orog[select_name(rlat_names, ds_orog.coords)].values
    dist_rlon=(rlon_values[0]- rlon_values[1])/2
    dist_rlat=(rlat_values[0]- rlat_values[1])/2 

    # sftlf mask
    dist = (ds_sftlf['lon']-lon)**2 + (ds_sftlf['lat']-lat)**2
    [ilat], [ilon] = np.where(dist == np.min(dist))
    ds_sftlf = ds_sftlf.isel(**{
    select_name(rlat_names, ds_sftlf.coords): slice(ilat-dlat,ilat+dlat),
    select_name(rlon_names, ds_sftlf.coords): slice(ilon-dlon,ilon+dlon)
    })
    sftlf_mask = ds_sftlf > sftlf_th   

    non_urban_mask=select_non_urban_cells(urban_mask,orog_mask,sftlf_mask,urban_surrounding_mask,scale)
    if save_results == True: 
        # Convert to dataset and create attribtes
        urban_area = urban_mask.astype(int).where(urban_mask.astype(int) == 1, np.nan)
        urban_area = urban_area.where(non_urban_mask.astype(int) == 0, 0)
        urban_area = urban_area.to_dataset(name='urban_area')
        
        urban_area['urban_area'].attrs['long_name'] = 'Urban vs. vicinity. 1 corresponds to urban areas and 0 to the surrounding areas'
        urban_area['urban_area'].attrs['lon_city'] = lon
        urban_area['urban_area'].attrs['lat_city'] = lat
        urban_area['urban_area'].attrs['urban_th'] = urban_th
        urban_area['urban_area'].attrs['urban_surrounding_th'] = urban_surrounding_th
        urban_area['urban_area'].attrs['orog_diff'] = orog_diff
        urban_area['urban_area'].attrs['landsea_th'] = sftlf_th
        urban_area['urban_area'].attrs['scale'] = scale
        urban_area['urban_area'].attrs['lon_lim'] = lon_lim
        urban_area['urban_area'].attrs['lat_lim'] = lat_lim
    
        urban_area.to_netcdf('urban_area.nc')
        plot_urban_areas(ds_urban,ds_orog,ds_sftlf,urban_mask,non_urban_mask,orog_mask, sftlf_mask,urban_th,urban_elevation,orog_diff,sftlf_th,urban_area, scale)

    # Plot contrast between urban areas 
    tn_mean=ds['tasmin'].isel(**{
    select_name(rlat_names, ds_sftlf.coords): slice(ilat-dlat,ilat+dlat),
    select_name(rlon_names, ds_sftlf.coords): slice(ilon-dlon,ilon+dlon)
    }).where(urban_mask).mean().compute()
    change=ds['tasmin'].isel(**{
    select_name(rlat_names, ds_sftlf.coords): slice(ilat-dlat,ilat+dlat),
    select_name(rlon_names, ds_sftlf.coords): slice(ilon-dlon,ilon+dlon)
    }).mean(dim=['time'])-tn_mean
    change.plot(ax=axes, transform=proj, cmap='seismic', vmin=vtmin,vmax=vtmax)
    ucdb_city.plot(ax=axes, facecolor="none", transform=lonlat, edgecolor="Green")
    axes.set_title(f'ºC')
    axes.coastlines()
    #for index, station in rval.iterrows():
    #    station_lon = station['lon']
    #    station_lat = station['lat']
    #    
    #    urban_mask_value = urban_mask.sel(lon=station_lon, lat=station_lat, method='nearest').item()
    #    sea_mask_value = sftlf_mask.sel(lon=station_lon, lat=station_lat, method='nearest').item()
    #    orog_mask_value=orog_mask.sel(lon=station_lon, lat=station_lat, method='nearest').item()
    #    
    #    if urban_mask_value > 0:
    #        station_color = 'grey'  
    #    elif sea_mask_value > 0 and orog_mask_value > 0:
    #        station_color = 'black'
    #    else:
    #        station_color = 'none' 
    #                
    #    axes[row, col].plot(station_lon, station_lat, marker='o', markersize=3, color=station_color)
    
    for i in range(len(rlon_values) ):
        for j in range(len(rlat_values)):
            if non_urban_mask[j, i].all()==True:
                plot_boundary(axes, rlon_values[i], rlat_values[j], dist_rlon, dist_rlat,'blue',zorder=5,linewidth=4)
            elif urban_mask[j, i].any() == True:
                plot_boundary(axes, rlon_values[i], rlat_values[j], dist_rlon, dist_rlat,'red',zorder=50,linewidth=4)

    # Time series plot
    # Calculate monthly mean temperature within urban mask
    time_urban = (ds['tasmin'].where(urban_mask).groupby('time.month').mean(dim=['time',select_name(rlat_names, ds_urban.coords),select_name(rlon_names, ds_urban.coords)]).compute())

    # Calculate anomaly (difference from overall mean) for urban time series
    time_plot_urban = time_urban - time_urban 
    
    # Plot anomaly for urban time series with percentile
    time_plot_urban.plot(ax=axes2, color='red', linewidth=3, linestyle='--',label='Mean of the urban cells')        
    time_plot_urban_percentil = (ds['tasmin'].where(urban_mask).groupby('time.month').mean(dim=['time'])- time_urban)
    lower_percentile_urban = np.nanpercentile(time_plot_urban_percentil, percentil, axis=[time_plot_urban_percentil.get_axis_num(select_name(rlat_names, ds_urban.coords)), time_plot_urban_percentil.get_axis_num(select_name(rlon_names, ds_urban.coords))])
    upper_percentile_urban = np.nanpercentile(time_plot_urban_percentil, 100-percentil, axis=[time_plot_urban_percentil.get_axis_num(select_name(rlat_names, ds_urban.coords)), time_plot_urban_percentil.get_axis_num(select_name(rlon_names, ds_urban.coords))])
    axes2.fill_between(time_plot_urban_percentil['month'], lower_percentile_urban, upper_percentile_urban, color='red', alpha=0.1)
    axes2.fill_between(time_plot_urban_percentil['month'], time_plot_urban_percentil.min(dim=[select_name(rlat_names, ds_urban.coords),select_name(rlon_names, ds_urban.coords)]), time_plot_urban_percentil.max(dim=[select_name(rlat_names, ds_urban.coords),select_name(rlon_names, ds_urban.coords)]), color='red', alpha=0.1)

    # Calculate monthly mean temperature for non-urban (rural) areas
    time_rural = (ds['tasmin'].where(~urban_mask).where(non_urban_mask==1).groupby('time.month').mean(dim=['time',select_name(rlat_names, ds_urban.coords),select_name(rlon_names, ds_urban.coords)]).compute())
    time_plot_rural = time_rural - time_urban

     # Plot anomaly for rural time series with percentiles
    time_plot_rural.plot(ax=axes2, color='blue', linewidth=3, linestyle='--', label='Mean of the not urban cells')        
    time_plot_rural_percentil = (ds['tasmin'].where(~urban_mask).where(non_urban_mask==1).groupby('time.month').mean(dim=['time']) - time_urban)
    lower_percentile_rural = np.nanpercentile(time_plot_rural_percentil, percentil, axis=[time_plot_rural_percentil.get_axis_num(select_name(rlat_names, ds_urban.coords)), time_plot_rural_percentil.get_axis_num(select_name(rlon_names, ds_urban.coords))])
    upper_percentile_rural = np.nanpercentile(time_plot_rural_percentil, 100-percentil, axis=[time_plot_rural_percentil.get_axis_num(select_name(rlat_names, ds_urban.coords)), time_plot_rural_percentil.get_axis_num(select_name(rlon_names, ds_urban.coords))])
    axes2.fill_between(time_plot_rural_percentil['month'], lower_percentile_rural, upper_percentile_rural, color='blue', alpha=0.1)
    axes2.fill_between(time_plot_rural_percentil['month'], time_plot_rural_percentil.min(dim=[select_name(rlat_names, ds_urban.coords),select_name(rlon_names, ds_urban.coords)]), time_plot_rural_percentil.max(dim=[select_name(rlat_names, ds_urban.coords),select_name(rlon_names, ds_urban.coords)]), color='blue', alpha=0.1)
    
    #Calculate and plot anomaly for individual cells
    for i in range(len(rlon_values)):
        for j in range(len(rlat_values)):
            if urban_mask[j, i].any():
                if select_name(rlat_names, ds_urban.coords)=='y':
                    time_plot_urban_loc = (ds['tasmin'].sel(x=rlon_values[i], y=rlat_values[j]).groupby('time.month').mean(dim=['time'])- time_urban)
                else:
                    time_plot_urban_loc = (ds['tasmin'].sel(rlon=rlon_values[i], rlat=rlat_values[j]).groupby('time.month').mean(dim=['time'])- time_urban)
                time_plot_urban_loc.plot(ax=axes2, color='red', linewidth=0.5)
            elif non_urban_mask[j, i].all()==1:
                if select_name(rlat_names, ds_urban.coords)=='y':
                    time_plot_rural_loc = (ds['tasmin'].sel(x=rlon_values[i], y=rlat_values[j]).groupby('time.month').mean(dim=['time'])- time_urban)
                else:
                    time_plot_rural_loc = (ds['tasmin'].sel(rlon=rlon_values[i], rlat=rlat_values[j]).groupby('time.month').mean(dim=['time'])- time_urban)

                time_plot_rural_loc.plot(ax=axes2, color='blue', linewidth=0.5)
    
    # Define month names list
    months_names = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December' ];
    
    # Add empty plot lines with labels for legend
    axes2.plot([], color='red', linewidth=0.5, label='Urban Cells')
    axes2.plot([], color='blue', linewidth=0.5, label='Not Urban Cells')
    
    # Set x-axis ticks and labels based on season
    
    axes2.set_xticks(range(1, 13))
    axes2.set_xticklabels(months_names)
    
    
    # Set axis labels, title and legend
    axes2.set_xlabel('Months')
    axes2.set_ylabel('Temperature Anomaly (°C)') 
    axes2.set_title('Monthly Temperature Anomalies (Urban vs. not Urban)')
    axes2.legend()

    return fig, fig2

In [ ]:
[ds,ds_sftlf, ds_orog, ds_urban, lat, lon, rval,res]=data_city("Athens",location, "REMO",model_dict,"CORDEX-CORE")

In [ ]:
domain='EUR-11'

In [ ]:
# Searching for files
file_sea=glob.glob('/lustre/gmeteo/DATA/ESGF/REPLICA/DATA/cordex/output/'+domain+'/GERICS/ECMWF-ERAINT/evaluation/r0i0p0/REMO2015/v1/fx/sftlf/v'+'*'+'/sftlf_'+domain+'*')
file_orog = glob.glob('/lustre/gmeteo/DATA/ESGF/REPLICA/DATA/cordex/output/'+domain+'/GERICS/ECMWF-ERAINT/evaluation/r0i0p0/REMO2015/v1/fx/orog/v'+'*'+'/orog_'+domain+'*')
file_urban=glob.glob('/lustre/gmeteo/WORK/DATA/CORDEX-FPS-URB-RCC/nextcloud/CORDEX-CORE-WG/REMO/urbanfraction/orig_v3/'+domain+'*')

sea_mask = xr.open_dataset(file_sea[0])
urbanfraction=xr.open_dataset(file_urban[0])
orography = xr.open_dataset(file_orog[0])

res = int(orography.attrs['CORDEX_domain'].split('-')[1])

ds_urban = urbanfraction['urban']
ds_orog = orography['orog']
ds_sftlf = sea_mask['sftlf'] 

In [ ]:
directory='/lustre/gmeteo/DATA/ESGF/REPLICA/DATA/cordex/output/'+domain+'/GERICS/ECMWF-ERAINT/evaluation/r1i1p1/REMO2015/v1/day/tasmin/v'
file_names = [
    'tasmin_'+domain+'_ECMWF-ERAINT_evaluation_r1i1p1_GERICS-REMO2015_v1_day_19790102-19801231.nc',
    'tasmin_'+domain+'_ECMWF-ERAINT_evaluation_r1i1p1_GERICS-REMO2015_v1_day_19910101-19951231.nc',
    'tasmin_'+domain+'_ECMWF-ERAINT_evaluation_r1i1p1_GERICS-REMO2015_v1_day_19960101-20001231.nc',
]
# Open each file individually and store them in a list
individual_datasets = []
for file_name in file_names:
    file_path = glob.glob(directory+'*/'+file_name)
    individual_datasets.append(xr.open_dataset(file_path[0]))

# Combine the individual datasets into a single dataset
ds = xr.concat(individual_datasets, dim='time')


In [ ]:
[plot1,plot2]=plot_cities(ds, "Athens", "REMO", model_dict, ds_sftlf, ds_orog, ds_urban,"CORDEX-CORE", lat, lon, 1,1, -2, 2, rval,rlat_names,rlon_names, rotated_pole_names,res)

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

# Crear un nuevo archivo PDF
with PdfPages("Athens-EUR-11-tasmin-RegCM-evaluation.pdf") as pdf:
    # Guardar el primer plot en el PDF
    pdf.savefig(plot1)
    # Guardar el segundo plot en el PDF
    pdf.savefig(plot2)


In [ ]:
directory='/lustre/gmeteo/DATA/ESGF/REPLICA/DATA/cordex/output/EAS-22/ICTP/MOHC-HadGEM2-ES/historical/r1i1p1/RegCM4-4/v0/day/tasmin/v20190502'
file_names = [
    'tasmin_EAS-22_MOHC-HadGEM2-ES_historical_r1i1p1_ICTP-RegCM4-4_v0_day_19800101-19801230.nc',
    'tasmin_EAS-22_MOHC-HadGEM2-ES_historical_r1i1p1_ICTP-RegCM4-4_v0_day_19810101-19811230.nc',
    'tasmin_EAS-22_MOHC-HadGEM2-ES_historical_r1i1p1_ICTP-RegCM4-4_v0_day_19820101-19821230.nc',
    'tasmin_EAS-22_MOHC-HadGEM2-ES_historical_r1i1p1_ICTP-RegCM4-4_v0_day_19830101-19831230.nc',
    'tasmin_EAS-22_MOHC-HadGEM2-ES_historical_r1i1p1_ICTP-RegCM4-4_v0_day_19840101-19841230.nc',
    'tasmin_EAS-22_MOHC-HadGEM2-ES_historical_r1i1p1_ICTP-RegCM4-4_v0_day_19850101-19851230.nc',
    'tasmin_EAS-22_MOHC-HadGEM2-ES_historical_r1i1p1_ICTP-RegCM4-4_v0_day_19860101-19861230.nc',
    'tasmin_EAS-22_MOHC-HadGEM2-ES_historical_r1i1p1_ICTP-RegCM4-4_v0_day_19870101-19871230.nc',
    'tasmin_EAS-22_MOHC-HadGEM2-ES_historical_r1i1p1_ICTP-RegCM4-4_v0_day_19880101-19881230.nc',
    'tasmin_EAS-22_MOHC-HadGEM2-ES_historical_r1i1p1_ICTP-RegCM4-4_v0_day_19890101-19891230.nc',
    'tasmin_EAS-22_MOHC-HadGEM2-ES_historical_r1i1p1_ICTP-RegCM4-4_v0_day_19900101-19901230.nc',
    'tasmin_EAS-22_MOHC-HadGEM2-ES_historical_r1i1p1_ICTP-RegCM4-4_v0_day_19910101-19911230.nc',
    'tasmin_EAS-22_MOHC-HadGEM2-ES_historical_r1i1p1_ICTP-RegCM4-4_v0_day_19920101-19921230.nc',
    'tasmin_EAS-22_MOHC-HadGEM2-ES_historical_r1i1p1_ICTP-RegCM4-4_v0_day_19930101-19931230.nc',
    'tasmin_EAS-22_MOHC-HadGEM2-ES_historical_r1i1p1_ICTP-RegCM4-4_v0_day_19940101-19941230.nc',
    'tasmin_EAS-22_MOHC-HadGEM2-ES_historical_r1i1p1_ICTP-RegCM4-4_v0_day_19950101-19951230.nc',
    'tasmin_EAS-22_MOHC-HadGEM2-ES_historical_r1i1p1_ICTP-RegCM4-4_v0_day_19960101-19961230.nc',
    'tasmin_EAS-22_MOHC-HadGEM2-ES_historical_r1i1p1_ICTP-RegCM4-4_v0_day_19970101-19971230.nc',
    'tasmin_EAS-22_MOHC-HadGEM2-ES_historical_r1i1p1_ICTP-RegCM4-4_v0_day_19980101-19981230.nc',
    'tasmin_EAS-22_MOHC-HadGEM2-ES_historical_r1i1p1_ICTP-RegCM4-4_v0_day_19990101-19991230.nc',
    'tasmin_EAS-22_MOHC-HadGEM2-ES_historical_r1i1p1_ICTP-RegCM4-4_v0_day_20000101-20001230.nc',
]

# Open each file individually and store them in a list
individual_datasets = []
for file_name in file_names:
    file_path = glob.glob(directory+'/'+file_name)
    individual_datasets.append(xr.open_dataset(file_path[0]))

# Combine the individual datasets into a single dataset
ds = xr.concat(individual_datasets, dim='time')


In [ ]:
# Searching for files
directory_orog ='/lustre/gmeteo/DATA/ESGF/REPLICA/DATA/cordex/output/EAS-22/ICTP/MOHC-HadGEM2-ES/historical/r1i1p1/RegCM4-4/v0/fx/orog/v20190502/orog_EAS-22_MOHC-HadGEM2-ES_historical_r1i1p1_ICTP-RegCM4-4_v0_fx.nc'
directory_sea = '/lustre/gmeteo/DATA/ESGF/REPLICA/DATA/cordex/output/EAS-22/ICTP/MOHC-HadGEM2-ES/historical/r1i1p1/RegCM4-4/v0/fx/sftlf/v20190502/sftlf_EAS-22_MOHC-HadGEM2-ES_historical_r1i1p1_ICTP-RegCM4-4_v0_fx.nc'
directory_urban='/lustre/gmeteo/WORK/DATA/CORDEX-FPS-URB-RCC/nextcloud/CORDEX-CORE-WG/RegCM/urbanfraction/EAS-22.nc'
sea_mask = xr.open_dataset(directory_sea)
urbanfraction=xr.open_dataset(directory_urban)
orography = xr.open_dataset(directory_orog)

res = int(orography.attrs['CORDEX_domain'].split('-')[1])

                 
ds_urban = urbanfraction['sftuf']
ds_orog = orography['orog']
ds_sftlf = sea_mask['sftlf'] 